In [1]:
import sys
# Add parent directory to the module search path
sys.path.append('../')

In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, ConcatDataset
from torch.utils.data.dataset import random_split
from torchvision import transforms

In [3]:
# load the data
facialpoints_df = pd.read_csv('../data/KeyFacialPoints.csv')

In [4]:
class FacialKeypointsDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        image_string = self.dataframe.iloc[idx, -1]
        image = np.array([int(item) for item in image_string.split()]).reshape(96, 96, 1)
        image = image / 255.
        keypoints = self.dataframe.iloc[idx, :-1].values.astype('float32')

        sample = {'image': image, 'keypoints': keypoints}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [5]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, keypoints = sample['image'], sample['keypoints']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1)).copy()  # Added .copy()

        return {'image': torch.from_numpy(image),
                'keypoints': torch.from_numpy(keypoints)}



class CustomHorizontalFlip:
    def __call__(self, sample):
        image, keypoints = sample['image'], sample['keypoints']
        image = image[:, ::-1, :]
        keypoints = keypoints.copy()
        keypoints[::2] = 96. - keypoints[::2]  # Assuming image size is 96x96 and keypoints are scaled accordingly
        return {'image': image, 'keypoints': keypoints}


class FlipVertical:
    def __call__(self, sample):
        image, keypoints = sample['image'], sample['keypoints']
        image = image[::-1, :, :]
        keypoints = keypoints.copy()
        keypoints[1::2] = 96. - keypoints[1::2]  # Assuming image size is 96x96 and keypoints are scaled accordingly
        return {'image': image, 'keypoints': keypoints}


class IncreaseBrightness:
    def __init__(self, value):
        self.value = value

    def __call__(self, sample):
        image, keypoints = sample['image'], sample['keypoints']
        image = np.clip(image + self.value, 0., 1.)  # Ensure values are within [0, 1]
        return {'image': image, 'keypoints': keypoints}


class DecreaseBrightness:
    def __init__(self, value):
        self.value = value

    def __call__(self, sample):
        image, keypoints = sample['image'], sample['keypoints']
        image = np.clip(image - self.value, 0., 1.)  # Ensure values are within [0, 1]
        return {'image': image, 'keypoints': keypoints}


In [6]:
# Define transformations
transforms_list = [
    transforms.Compose([CustomHorizontalFlip(), ToTensor()]),
    transforms.Compose([FlipVertical(), ToTensor()]),
    transforms.Compose([IncreaseBrightness(0.1), ToTensor()]),
    transforms.Compose([DecreaseBrightness(0.1), ToTensor()])
]

# Create the original dataset
datasets = [FacialKeypointsDataset(dataframe=facialpoints_df, transform=ToTensor())]

# Apply each transform to the original data and append to the datasets list
for transform in transforms_list:
    transformed_dataset = FacialKeypointsDataset(dataframe=facialpoints_df, transform=transform)
    datasets.append(transformed_dataset)

# Combine original and transformed datasets
combined_dataset = ConcatDataset(datasets)

In [7]:
# Determine lengths of splits
total_length = len(combined_dataset)
train_length = int(0.8 * total_length)  # 80% of the total data
test_length = total_length - train_length  # 20% of the total data
val_length = int(0.1 * train_length)  # 10% of the training data
train_length = train_length - val_length  # Adjust training data length

# Split data
train_data, test_data = random_split(combined_dataset, [train_length + val_length, test_length])  # First, split into train+val and test
train_data, val_data = random_split(train_data, [train_length, val_length])  # Then, split train into train and val

# Print length of datasets
print(f"Length of training set: {len(train_data)}")
print(f"Length of validation set: {len(val_data)}")
print(f"Length of test set: {len(test_data)}")

Length of training set: 7704
Length of validation set: 856
Length of test set: 2140


In [8]:
train_dict = next(iter(train_data))
val_dict = next(iter(val_data))
test_dict = next(iter(test_data))

print(f"Shape of training sample: {train_dict['image'].shape}, Shape of training label: {train_dict['keypoints'].shape}")
print(f"Shape of validation sample: {val_dict['image'].shape}, Shape of validation label: {val_dict['keypoints'].shape}")
print(f"Shape of test sample: {test_dict['image'].shape}, Shape of test label: {test_dict['keypoints'].shape}")


Shape of training sample: torch.Size([1, 96, 96]), Shape of training label: torch.Size([30])
Shape of validation sample: torch.Size([1, 96, 96]), Shape of validation label: torch.Size([30])
Shape of test sample: torch.Size([1, 96, 96]), Shape of test label: torch.Size([30])


In [9]:
# save datasets
torch.save(train_data, '../data/train_data.pth')
torch.save(val_data, '../data/val_data.pth')
torch.save(test_data, '../data/test_data.pth')